In [1]:
import os

In [2]:
os.getcwd()

'/Users/abishekkumaryadav/DataScience/Machine_learning/Customer_segmentation_project/research'

In [3]:
os.chdir('../')

In [33]:
from dataclasses import dataclass
from src.Customer_segementation.logger import logger
from src.Customer_segementation.constant import *

In [34]:
@dataclass
class DataIngestionConfig: 
    root_dir: Path
    data_path: Path
    mongodb_url: str
    database_name: str
    collection_name: str

In [35]:
from src.Customer_segementation.utils.common import read_yaml, create_directories

In [36]:
class configurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, 
                 params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            mongodb_url=config.mongodb_url,
            database_name=config.DATABASE_NAME,
            collection_name=config.COLLECTION_NAME

        )
        return data_ingestion_config

In [37]:
from pymongo.mongo_client import MongoClient
import pandas as pd

In [38]:
from pymongo import MongoClient
import ssl

In [39]:
class DataIngestion: 
    def __init__(self,config=DataIngestionConfig):
        self.config=config


    def import_collection_as_DataFrame(self, database_name: str,
                                        collection_name: str, mongodb_uri: str):
        try: 
            client=MongoClient(mongodb_uri, tls=True, tlsAllowInvalidCertificates=True)

            collection_name=client[database_name][collection_name]

            data=pd.DataFrame(list(collection_name.find()))
            
            for col in data.columns: 
                if col =='_id':
                    data.drop('_id', axis=1, inplace=True)
            
            return data


        except Exception as e:
            raise e

    def import_data_into_feature_store_file_system(self):
        """
        Method_name: Import data into feature store file.
        Description: This method import data from mongodb and save into artifacts folder.

        outPut: data return as DataFrame
        onfailure: Write exception log and raise exception.
        """

        try: 
            logger.info("Importing DataFrame from Mongodb")
            data=self.import_collection_as_DataFrame(database_name=self.config.database_name,
                                                collection_name=self.config.collection_name,
                                                mongodb_uri=self.config.mongodb_url
                                                )
            
            data.to_csv(self.config.data_path, header=True, index=False)

        except Exception as e: 
            raise e


In [40]:
try: 
    configmanger=configurationManager()
    dataingestion_config=configmanger.get_data_ingestion_config()
    data_ingestion=DataIngestion(dataingestion_config)
    data_ingestion.import_data_into_feature_store_file_system()
except Exception as e:
    raise e

[2024-07-01 16:50:33,299 : INFO : common : Yaml file read config/config.yaml successfully]
[2024-07-01 16:50:33,301 : INFO : common : Yaml file read params.yaml successfully]
[2024-07-01 16:50:33,302 : INFO : common : Directories created ['artifacts']]
[2024-07-01 16:50:33,302 : INFO : common : Directories created ['artifacts/data_ingestion']]
[2024-07-01 16:50:33,303 : INFO : 4116503278 : Importing DataFrame from Mongodb]
